# Applied Data Science Capstone

## Week 3 Assignment : Part 2 : Segmenting and Clustering Toronto Neighborhoods

### Author : Bill Gourley
### Date     : 29/11/2018

#### Description :  This notebook combines Part 1 of the assignment and adds the code to download a csv file containing
####                         latititude and longitude information for each Toronto Postcode and merges this information with the 
####                         cleaned dataframe produced in Part 1.

In [53]:
#import required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv

pd.options.display.max_columns = None
pd.options.display.max_rows = None

##### Download Wikipedia Page

In [54]:
result = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

##### Parse the result content into a BeautifulSoup object

In [55]:
soup = BeautifulSoup(result.content,'lxml')

##### Extract the table from the page (as there is only 1 table, we do not need to use find_all)

In [56]:
table = soup.find('table',{'class':'wikitable sortable'})

##### Extract the rows from the table and append to a list (each extracted row is also a list)

In [57]:
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

##### The first list entry is empty as we have not extracted the headers (as there is only 3, they will be added manually to the dataframe

In [58]:
#input row data
del(l[0])

##### Create a Pandas DataFrame from the extracted row list, and add the column names

In [59]:
#create dataframe
columns=['Postcode','Borough','Neighborhood']
df = pd.DataFrame(l,columns=columns)
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


##### Clean the Neighborhood column by removing the trailing newline string

In [60]:
#remove \n from Neighborhood column
df.Neighborhood = df.Neighborhood.str.replace('\n','')

##### There are 289 rows and 3 columns in the dataframe before further cleaning

In [61]:
#shape of dataframe before dropping rows
df.shape

(289, 3)

##### Clean dataframe by dropping any row which has a 'Not assigned' value for Borough

In [62]:
#drop all rows where Borough = 'Not assigned'
df = df.drop(df[df.Borough == 'Not assigned'].index)

##### This leaves 212 rows and 3 columns in the dataframe

In [63]:
#shape of dataframe after dropping rows
df.shape

(212, 3)

##### Join Neighborhood values with a comma where Postcode and Borough are identical and remove duplicate rows

In [64]:
#join Neighborhood values with a comma where Postcode and Borough are identical
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

##### Assign Neighborhood value to Borough where Neighborhood is 'Not assigned' and check there are no 'Not assigned' Neighborhoods remaining

In [65]:
#Assign Neighborhood value to Borough where Neighborhood is 'Not assigned'
df.Neighborhood = np.where(df.Neighborhood == 'Not assigned',df.Borough,df.Neighborhood)

#check there are no 'Not assigned' Neighborhoods remaining
print(df.loc[df.Neighborhood == 'Not assigned'].count())

Postcode        0
Borough         0
Neighborhood    0
dtype: int64


##### First 5 rows of the cleaned dataframe

In [66]:
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### The cleaned dataframe contains 103 rows and 3 columns

In [67]:
#shape of the cleaned dataframe
df.shape

(103, 3)

##### Download Geospatial_data csv file and extract contents into a list

In [68]:
# Download Geospatial_data csv file and extract contents into a list
csv_url = 'http://cocl.us/Geospatial_data'
with requests.Session() as s:
    download = s.get(csv_url)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    latlon_list = list(cr)

##### Remove first list element containing header information and create a dataframe with the contents of the list

In [69]:
# Remove first list element containing header information and create a dataframe with the contents of the list
del latlon_list[0]
columns = ['Postcode','Latitude','Longitude']
df_latlon = pd.DataFrame(latlon_list,columns = columns)

##### Check shape of the new datframe is the same as the cleaned dataframe

In [70]:
# Check shape of the new datframe is the same as the cleaned dataframe
df_latlon.shape

(103, 3)

##### Merge the latitude and longitude dataframe with the original cleaned dataframe 

In [71]:
# Merge the latitude and longitude dataframe with the original cleaned dataframe 
df_toronto = pd.merge(df, df_latlon, on='Postcode')

##### First 5 rows of merged dataframe

In [72]:
# First 5 rows of merged dataframe
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761
